<a href="https://colab.research.google.com/github/sultanardia/Twitter-NLP/blob/main/Twitter_NLP_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Dataset

## Import Dependencies

In [ ]:
!pip install sastrawi

     |████████████████████████████████| 209 kB 6.9 MB/s 


In [ ]:
import pandas as pd
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

# NLTK libraries
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Sastrawi library
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Instance the function

In [ ]:
def clean_multiple_space(series):
  return series.str.replace('\s+', ' ')

def clean_punctuation(series):
  return series.str.replace('[^\w\s-]', '')

def lowercase(series):
  return series.apply(lambda x : x.lower())

## Collecting dataset

### Read dataset

Instagram cyberbullying dataset

In [ ]:
df_ins_raw = pd.read_csv('drive/MyDrive/Dataset/Dataset-Sentimen-Analisis-Bahasa-Indonesia-master/dataset_komentar_instagram_cyberbullying.csv')
df_ins = df_ins_raw[['Sentiment', 'Instagram Comment Text']]
df_ins.columns = ['sentiment', 'raw_text']
df_ins.head()

,sentiment,raw_text
0,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,negative,Geblek lo tata...cowo bgt dibela2in balikan......
2,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."


TV dataset

In [ ]:
df_tv_raw = pd.read_csv('drive/MyDrive/Dataset/Dataset-Sentimen-Analisis-Bahasa-Indonesia-master/dataset_tweet_sentimen_tayangan_tv.csv')
df_tv = df_tv_raw[['Sentiment', 'Text Tweet']]
df_tv.columns = ['sentiment', 'raw_text']
df_tv.head()

,sentiment,raw_text
0,positive,"Undang @N_ShaniJKT48 ke hitamputih, pemenang S..."
1,positive,Selamat berbuka puasa Semoga amal ibadah hari ...
2,positive,"Ada nih di trans7 hitam putih, dia dpt penghar..."
3,positive,selamat ya mas @adietaufan masuk hitamputih
4,positive,Asiknya nonton Hitam Putih Trans7


Cellular provider dataset

In [ ]:
df_cell_raw = pd.read_csv('drive/MyDrive/Dataset/Dataset-Sentimen-Analisis-Bahasa-Indonesia-master/dataset_tweet_sentiment_cellular_service_provider.csv')
df_cell = df_cell_raw[['Sentiment', 'Text Tweet']]
df_cell.columns = ['sentiment', 'raw_text']
df_cell.head()

,sentiment,raw_text
0,positive,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...
1,positive,"Saktinya balik lagi, alhamdulillah :v <PROVIDE..."
2,negative,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...
3,negative,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...
4,negative,Selamat malam PENDUSTA <PROVIDER_NAME>


Film opinion dataset

In [ ]:
df_film_raw = pd.read_csv('drive/MyDrive/Dataset/Dataset-Sentimen-Analisis-Bahasa-Indonesia-master/dataset_tweet_sentiment_opini_film.csv')
df_film = df_film_raw[['Sentiment', 'Text Tweet']]
df_film.columns = ['sentiment', 'raw_text']
df_film.head()

,sentiment,raw_text
0,negative,Jelek filmnya... apalagi si ernest gak mutu bg...
1,negative,Film king Arthur ini film paling jelek dari se...
2,negative,@beexkuanlin Sepanjang film gwa berkata kasar ...
3,negative,Ane ga suka fast and furious..menurutku kok je...
4,negative,"@baekhyun36 kan gua ga tau film nya, lu bilang..."


Pilkada dataset

In [ ]:
df_pil_raw = pd.read_csv('drive/MyDrive/Dataset/Dataset-Sentimen-Analisis-Bahasa-Indonesia-master/dataset_tweet_sentiment_pilkada_DKI_2017.csv')
df_pil = df_pil_raw[['Sentiment', 'Text Tweet']]
df_pil.columns = ['sentiment', 'raw_text']
df_pil.head()

,sentiment,raw_text
0,negative,Banyak akun kloning seolah2 pendukung #agussil...
1,negative,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,negative,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,negative,Kasian oh kasian dengan peluru 1milyar untuk t...
4,negative,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


## Concatenate all dataframes

In [ ]:
df = pd.concat([df_ins, df_tv, df_cell, df_film, df_pil]).reset_index()
del df['index']
df

,sentiment,raw_text
0,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,negative,Geblek lo tata...cowo bgt dibela2in balikan......
2,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."
...,...,...
2195,positive,"Kali saja bpk @aniesbaswedan @sandiuno lihat, ..."
2196,positive,Kita harus dapat merangkul semua orang tanpa b...
2197,positive,Ini jagoanku dibidang digital <Smiling Face Wi...
2198,positive,#PesanBijak #OkeOce #GubernurGu3 ...


## Cleaning dataset

### Clean multiple spaces

In [ ]:
df['raw_text'] = clean_multiple_space(df['raw_text'])
df.head()

,sentiment,raw_text
0,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,negative,Geblek lo tata...cowo bgt dibela2in balikan......
2,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."


### Clean the punctuations

In [ ]:
df['raw_text'] = clean_punctuation(df['raw_text'])
df['raw_text'] = df['raw_text'].str.replace('_', '')
df.head()

,sentiment,raw_text
0,negative,USERNAME TOLOL Gak ada hubungan nya keguguran...
1,negative,Geblek lo tatacowo bgt dibela2in balikanhadeww...
2,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,negative,Intinya kalau kesel dengan ATT nya gausah ke a...
4,negative,hadewwwww permpuan itu lgsakit jiwaknp harus d...


### All lowercase

In [ ]:
df['raw_text'] = lowercase(df['raw_text'])
df.head()

,sentiment,raw_text
0,negative,username tolol gak ada hubungan nya keguguran...
1,negative,geblek lo tatacowo bgt dibela2in balikanhadeww...
2,negative,kmrn termewek2 skr lengket lg duhhh kok labil ...
3,negative,intinya kalau kesel dengan att nya gausah ke a...
4,negative,hadewwwww permpuan itu lgsakit jiwaknp harus d...


In [ ]:
df.to_csv('drive/MyDrive/Dataset/Dataset-Sentimen-Analisis-Bahasa-Indonesia-master/dataset_full.csv')

## Stopword

### Stopword by NLTK


In [ ]:
nltk_stopwords = stopwords.words('indonesian')

In [ ]:
df['nltk_stp'] = df['raw_text'].apply(lambda x : ' '.join(x for x in x.split() if x not in nltk_stopwords))
df.head()

,sentiment,raw_text,nltk_stp
0,negative,username tolol gak ada hubungan nya keguguran...,username tolol gak hubungan nya keguguran dgn ...
1,negative,geblek lo tatacowo bgt dibela2in balikanhadeww...,geblek lo tatacowo bgt dibela2in balikanhadeww...
2,negative,kmrn termewek2 skr lengket lg duhhh kok labil ...,kmrn termewek2 skr lengket lg duhhh labil bgt ...
3,negative,intinya kalau kesel dengan att nya gausah ke a...,intinya kesel att nya gausah anaknya kasian pe...
4,negative,hadewwwww permpuan itu lgsakit jiwaknp harus d...,hadewwwww permpuan lgsakit jiwaknp yg jd peran...


### Stopword by Sastrawi

In [ ]:
sastrawi = StopWordRemoverFactory()
sastrawi_stopwords = sastrawi.create_stop_word_remover()

In [ ]:
df['sastrawi_stp'] = df['raw_text'].apply(lambda x : ' '.join(sastrawi_stopwords.remove(x) for x in x.split()))
df.head()

,sentiment,raw_text,nltk_stp,sastrawi_stp
0,negative,username tolol gak ada hubungan nya keguguran...,username tolol gak hubungan nya keguguran dgn ...,username tolol gak hubungan nya keguguran dgn...
1,negative,geblek lo tatacowo bgt dibela2in balikanhadeww...,geblek lo tatacowo bgt dibela2in balikanhadeww...,geblek lo tatacowo bgt dibela2in balikanhadeww...
2,negative,kmrn termewek2 skr lengket lg duhhh kok labil ...,kmrn termewek2 skr lengket lg duhhh labil bgt ...,kmrn termewek2 skr lengket lg duhhh kok labil ...
3,negative,intinya kalau kesel dengan att nya gausah ke a...,intinya kesel att nya gausah anaknya kasian pe...,intinya kalau kesel att nya gausah anaknya ...
4,negative,hadewwwww permpuan itu lgsakit jiwaknp harus d...,hadewwwww permpuan lgsakit jiwaknp yg jd peran...,hadewwwww permpuan lgsakit jiwaknp yg jd pe...


In [ ]:
df.to_csv('drive/MyDrive/Dataset/Dataset-Sentimen-Analisis-Bahasa-Indonesia-master/dataset_full.csv')

## Pre-processing dataset

### Encoding label


In [ ]:
LE = LabelEncoder()

df['encoded label'] = LE.fit_transform(df['sentiment'])
df.head()

,sentiment,raw_text,nltk_stp,sastrawi_stp,encoded label
0,negative,username tolol gak ada hubungan nya keguguran...,username tolol gak hubungan nya keguguran dgn ...,username tolol gak hubungan nya keguguran dgn...,0
1,negative,geblek lo tatacowo bgt dibela2in balikanhadeww...,geblek lo tatacowo bgt dibela2in balikanhadeww...,geblek lo tatacowo bgt dibela2in balikanhadeww...,0
2,negative,kmrn termewek2 skr lengket lg duhhh kok labil ...,kmrn termewek2 skr lengket lg duhhh labil bgt ...,kmrn termewek2 skr lengket lg duhhh kok labil ...,0
3,negative,intinya kalau kesel dengan att nya gausah ke a...,intinya kesel att nya gausah anaknya kasian pe...,intinya kalau kesel att nya gausah anaknya ...,0
4,negative,hadewwwww permpuan itu lgsakit jiwaknp harus d...,hadewwwww permpuan lgsakit jiwaknp yg jd peran...,hadewwwww permpuan lgsakit jiwaknp yg jd pe...,0


### Split dataset

In [ ]:
train, test = train_test_split(df, test_size = 0.2)
print('Train size : ', train.shape)
print('Test size : ', test.shape)

Train size :  (1760, 5)
Test size :  (440, 5)


## SVM Classifier

Create x_train and y_train

In [ ]:
# x_train
count_vect = CountVectorizer()
x_vect = count_vect.fit_transform(train['raw_text'])
tfidf = TfidfTransformer()
x_train = tfidf.fit_transform(x_vect)

# y_train
y_train = train['encoded label']

Create x_test and y_test

In [ ]:
# x_test
x = count_vect.transform(test['raw_text'])
x_test = tfidf.transform(x)

# y_test
y_test = test['encoded label']

Generate SVM model

In [ ]:
svc = SVC(kernel = 'linear')
svc.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Testing and scoring

In [ ]:
svm_pred = svc.predict(x_test)

In [ ]:
print('Final accuracy score : {}'.format(accuracy_score(svm_pred, y_test) * 100))

Final accuracy score : 77.95454545454545
